## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [11]:
%%sql
postgresql:///animal_shelters

-- A CTE that contains all important variables
WITH query AS (SELECT CASE
    WHEN a.animaltype = 'Dog' AND weight <= 10 THEN 'DS'
    WHEN a.animaltype = 'Dog' AND weight > 10 AND weight <= 30 THEN 'DM'
    WHEN a.animaltype = 'Dog' AND weight > 30 THEN 'DL'
    WHEN a.animaltype = 'Cat' AND weight <= 5 THEN 'CS'
    WHEN a.animaltype = 'Cat' AND weight > 5 AND weight <= 7 THEN 'CM'
    WHEN a.animaltype = 'Cat' AND weight > 5 THEN 'CL'
    WHEN a.animaltype = 'Bird' AND weight <= 0.7 THEN 'BS'
    WHEN a.animaltype = 'Bird' AND weight > 0.7 AND weight <= 1.1 THEN 'BM'
    WHEN a.animaltype = 'Bird' AND weight > 1.1 THEN 'BL'
END AS sizeid,
animalid,
a.animaltype,
a.location,
costs AS location_costs,
DATE_PART('year', '2021-12-31'::date) - DATE_PART('year', date(birthdate)) AS age
FROM animals AS a
JOIN location_costs AS l
ON a.location = l.location
LEFT JOIN sponsored_animals AS sa
-- Counting out sponsored animals
ON a.animalid = sa.sponsorid         
WHERE sa.sponsorid IS NULL
)

SELECT query.animaltype, 
size_costs.size,
(SUM(age_costs.costs)+SUM(location_costs)+SUM(size_costs.costs)) AS total,
ROUND((SUM(age_costs.costs)+SUM(location_costs)+SUM(size_costs.costs)) * 100.0 / SUM(SUM(age_costs.costs)+SUM(location_costs)+SUM(size_costs.costs)) OVER(),2) AS percentage 
FROM query
JOIN size_costs
ON query.sizeid = size_costs.sizeid
JOIN age_costs
ON query.age = age_costs.age::NUMERIC
GROUP BY query.sizeid, query.animaltype, size_costs.size
ORDER BY query.animaltype, size_costs.size DESC

9 rows affected.


animaltype,size,total,percentage
Bird,Small,1615,0.05
Bird,Medium,3130,0.09
Bird,Large,8120,0.23
Cat,Small,518015,14.90
Cat,Medium,250575,7.21
Cat,Large,439490,12.64
Dog,Small,336530,9.68
Dog,Medium,941895,27.09
Dog,Large,977665,28.12
